In [1]:
from keras.layers import Conv2D, UpSampling2D, InputLayer
from keras.models import Sequential
from keras.preprocessing.image import img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np

# Load and preprocess the image

# Build the model
model = Sequential([
    InputLayer(input_shape=(None, None, 1)),
    Conv2D(8, (3, 3), activation='relu', padding='same', strides=2),
    Conv2D(8, (3, 3), activation='relu', padding='same'),
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    Conv2D(32, (3, 3), activation='relu', padding='same', strides=2),
    UpSampling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(2, (3, 3), activation='tanh', padding='same')
])

In [2]:
image = img_to_array(load_img('D:\stc\Coloring-greyscale-images\Alpha-version\man.jpg'))
image = np.array(image, dtype=float) / 255.0  # Normalize to range [0, 1]

# Convert image to LAB color space
X = rgb2lab(image)[:, :, 0]  # L channel
Y = rgb2lab(image)[:, :, 1:] / 128.0  # AB channels

# Reshape input for model
X = X.reshape(1, X.shape[0], X.shape[1], 1)
Y = Y.reshape(1, Y.shape[0], Y.shape[1], 2)


In [3]:
# Compile and train the model
model.compile(optimizer='rmsprop', loss='mse')
model.fit(x=X, y=Y, batch_size=1, epochs=1000)

# Predict and post-process the output
output = model.predict(X)
output *= 128

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2668
Epoch 2/1000
1/1 [==============================] - 0s 207ms/step - loss: 0.0180
Epoch 3/1000
1/1 [==============================] - 0s 247ms/step - loss: 0.0126
Epoch 4/1000
1/1 [==============================] - 0s 224ms/step - loss: 8.2500e-04
Epoch 5/1000
1/1 [==============================] - 0s 253ms/step - loss: 5.9999e-04
Epoch 6/1000
1/1 [==============================] - 0s 266ms/step - loss: 5.4216e-04
Epoch 7/1000
1/1 [==============================] - 0s 286ms/step - loss: 4.9522e-04
Epoch 8/1000
1/1 [==============================] - 0s 407ms/step - loss: 4.5272e-04
Epoch 9/1000
1/1 [==============================] - 0s 295ms/step - loss: 4.1360e-04
Epoch 10/1000
1/1 [==============================] - 0s 240ms/step - loss: 3.7775e-04
Epoch 11/1000
1/1 [==============================] - 0s 212ms/step - loss: 3.4505e-04
Epoch 12/1000
1/1 [==============================] - 0s 213ms/step - loss: 3.1

In [4]:
cur = np.zeros((400, 400, 3))
cur[:, :, 0] = X[0, :, :, 0] 
cur[:, :, 1:] = output[0]  # Assign the predicted AB channels

# Convert to uint8 and scale to [0, 255] range
colorized_image_uint8 = (image * 255).astype(np.uint8)

# Save the colorized image
imsave("img_result.png", colorized_image_uint8)

# Convert colorized image to grayscale
grayscale_image = rgb2gray(image)

# Convert grayscale image to 8-bit grayscale
grayscale_image_uint8 = (grayscale_image * 255).astype(np.uint8)

# Save the grayscale image
imsave("img_gray_version.png", grayscale_image_uint8)
